In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Stable Diffusion XL 1.0 - Latent Consistent Model

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_xl_lcm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_xl_lcm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_xl_lcm.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
    (a Python-3 GPU notebook with preinstalled HuggingFace/transformer libraries is recommended)
  </td>
</table>

## Overview

This notebook demonstrates running local inference for both the [Latent Consistent Model (LCM)](https://huggingface.co/docs/diffusers/main/en/using-diffusers/inference_with_lcm) and [LCM-LoRA](https://huggingface.co/docs/diffusers/main/en/using-diffusers/inference_with_lcm_lora) based on the [stabilityai/stable-diffusion-xl-base-1.0](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0) models in Colab and deploying the model on Vertex AI for online prediction.

### Objective

- Run local predictions for `text-to-image` with LCM and LCM-LoRA.
- Deploy the model to a [Vertex AI Endpoint resource](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for text-to-image.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

**NOTE**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

### Setup notebook

#### Colab
Run the following commands for Colab.

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

! pip3 install --upgrade pip
! pip3 install --upgrade torch==2.1.0+cu121 -f https://download.pytorch.org/whl/torch_stable.html
! pip3 install transformers==4.35.0
! pip3 install peft==0.6.2
! pip3 install diffusers==0.23.1
! pip3 install accelerate==0.24.1
! pip3 install safetensors
# Remove wrong cublas version.
! pip3 uninstall nvidia_cublas_cu11 --yes

# Restart the notebook kernel after installs.
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Setup Google Cloud project

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) with `Vertex AI User` and `Storage Object Admin` roles for deploying models to Vertex AI endpoint.

Set following variables for experiments environment:

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in, such as "us-central1"
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

# The service account for deploying fine tuned model. Something like: "your-name-test@{PROJECT_ID}.iam.gserviceaccount.com"
SERVICE_ACCOUNT = ""

Initialize Vertex AI API:

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### Define constants

In [ ]:
# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve-opt:20240223_1230_RC00"

### Define common functions

In [ ]:
import base64
from io import BytesIO

from google.cloud import aiplatform
from PIL import Image


def base64_to_image(image_str):
    """Convert base64 encoded string to an image."""
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image


def image_grid(imgs, rows=2, cols=2):
    """Arrange and display multiple images in a grid."""
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


def deploy_model(model_id, task, refiner_model_id="none"):
    """Create a Vertex AI Endpoint and deploy the specified model to the endpoint."""
    model_name = "stable-diffusion"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-{task}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    machine_type = "g2-standard-8"
    accelerator_type = "NVIDIA_L4"

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=1,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
    )
    return model, endpoint

## Run inferences locally

### Text-to-image with LCM

In [ ]:
import torch
from diffusers import (LCMScheduler, StableDiffusionXLPipeline,
                       UNet2DConditionModel)
from diffusers.models.attention_processor import AttnProcessor2_0

torch.cuda.empty_cache()

unet = UNet2DConditionModel.from_pretrained(
    "latent-consistency/lcm-sdxl",
    torch_dtype=torch.float16,
    variant="fp16",
)
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    unet=unet,
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

pipe.upcast_vae()
pipe.unet.set_attn_processor(AttnProcessor2_0())
pipe.unet.to(memory_format=torch.channels_last)

prompt = "a deer in Matisse style"

# We set `num_inference_steps` as 8 for a good image, feel free to use a different number of steps to match your needs.
# `guidance_scale` needs to be set as 0.
generator = torch.manual_seed(21)
image = pipe(
    prompt=prompt,
    num_inference_steps=8,
    generator=generator,
    guidance_scale=0.0,
).images[0]
display(image)

### Text-to-image with LCM-LoRA

In [ ]:
import torch
from diffusers import DiffusionPipeline, LCMScheduler
from diffusers.models.attention_processor import AttnProcessor2_0

torch.cuda.empty_cache()

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)

# set scheduler
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

pipe.upcast_vae()
pipe.unet.set_attn_processor(AttnProcessor2_0())
pipe.unet.to(memory_format=torch.channels_last)

# load LCM-LoRA
pipe.load_lora_weights("latent-consistency/lcm-lora-sdxl")

prompt = "a deer in Matisse style"

# We set `num_inference_steps` as 8 for a good image, feel free to use a different number of steps to match your needs.
# `guidance_scale` needs to be set as 0.
generator = torch.manual_seed(21)
image = pipe(
    prompt=prompt,
    num_inference_steps=8,
    generator=generator,
    guidance_scale=0.0,
).images[0]
display(image)

## Upload and Deploy models

This section uploads the model to Model Registry and deploys it to a Vertex AI Endpoint resource.

The model deployment step will take ~20 minutes to complete.

### Text-to-image with LCM

Deploy the stable diffusion XL LCM model for the text-to-image task.

Once deployed, you can send a batch of text prompts to the endpoint to generated images.

When deployed on one L4 GPU, the averaged inference time of a request is ~2-3 seconds.

In [ ]:
# Set the model_id to "stabilityai/stable-diffusion-xl-base-1.0" to load the OSS pre-trained model.
model, endpoint = deploy_model(
    model_id="stabilityai/stable-diffusion-xl-base-1.0", task="text-to-image-sdxl-lcm"
)

NOTE: The model weights will be downloaded after the deployment succeeds. Thus upto 10 minutes of additional waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

In [ ]:
instances = [
    {
        "prompt": "Photorealistic whale swimming in abyss",
        "height": 1024,
        "width": 1024,
        "num_inference_steps": 8,
    },
    {
        "prompt": "Photorealistic happy dog running",
        "height": 1024,
        "width": 1024,
        "num_inference_steps": 8,
    },
]
response = endpoint.predict(instances=instances)
images = [base64_to_image(image) for image in response.predictions]
image_grid(images, rows=1)

Clean up resources:

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()

### Text-to-image with LCM-LoRA

Deploy the base stable diffusion models, with LCM-LoRA.

When deployed on one L4 GPU, the averaged inference time of a request is ~3 seconds.

In [ ]:
# Set the model_id to "stabilityai/stable-diffusion-xl-base-1.0".
model, endpoint = deploy_model(
    model_id="stabilityai/stable-diffusion-xl-base-1.0",
    task="text-to-image-sdxl-lcm-lora",
)

NOTE: The model weights will be downloaded after the deployment succeeds. Thus additional **5 minutes** of waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.


In [ ]:
instances = [
    {"prompt": "a deer in Matisse style"},
    {"prompt": "a dog in Van Gogh style"},
]
response = endpoint.predict(instances=instances)
images = [base64_to_image(image) for image in response.predictions]
image_grid(images, rows=1)

Clean up resources:

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()